# Mistplay Fraud Demo – Model Serving

This notebook shows how to call a Databricks Model Serving endpoint for real-time fraud scoring. It reads the best model from the `model_selection` table (based on ROC AUC) so you can deploy that model in the UI.

**Suggested endpoint name:** `mistplay-fraud-serving`

**Payload tip:** because the model was logged with Feature Store, you only need to send the entity keys (`account_id`, `device_id`) and Feature Store will:
* Look up pre-computed features from online tables (account_features, device_features)
* **Automatically invoke the `last_batch_cutoff` UC function** to compute the on-demand feature at inference time
* Combine all features and return predictions

**No additional code needed** - the on-demand feature function is called automatically by the serving endpoint!

In [0]:
import os
import requests
from pyspark.sql import functions as F

DB_NAME = "ramin_serverless_aws_catalog.mistplay_fraud_demo"
ENDPOINT_NAME = "mistplay_fraud_demo_v6"


# best_model = (
#     spark.table(f"{DB_NAME}.model_selection")
#     .orderBy(F.col("selected_at").desc())
#     .select("model_name")
#     .first()
# )

# if not best_model:
#     raise ValueError("No model selection found. Run the training notebook first.")

model_name = 'ramin_serverless_aws_catalog.mistplay_fraud_demo.mistplay_fraud_model_lr'

print("Selected model:")
print(f"- {model_name}")

# Build a small request payload from existing entities
sample_entities = (
    spark.table(f"{DB_NAME}.account_device")
    .select("account_id", "device_id")
    .limit(5)
    .toPandas()
)

payload = {"dataframe_records": sample_entities.to_dict(orient="records")}



In [0]:
sample_entities.to_dict(orient='split')

In [0]:
type(payload)

In [0]:
# Get the current workspace host and token
host = "https://fevm-ramin-serverless-aws.cloud.databricks.com"
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

url = f"{host}/serving-endpoints/{ENDPOINT_NAME}/invocations"
headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}

response = requests.post(url, headers=headers, json=payload, timeout=30)

print("Status:", response.status_code)
print("Response:", response.text)

In [0]:
token

#### Example of using AI_Query.  Endpoint will retrieve any feature not supplied in query from online table

In [0]:
%sql
-- Note: ai_query() is returning null predictions
-- This is a known issue when Feature Store models return predictions in array format
-- The REST API (cell 5) works correctly and returns {"predictions": [0, 0, 0, 0, 0]}
-- For production use, prefer the REST API approach or use spark_ai_query() in Python

SELECT 
    account_id,
    device_id,
    is_fraud_label,  -- Ground truth from training data
    ai_query(
        endpoint => 'mistplay_fraud_demo_v6',
        request => named_struct(
            'account_id', account_id,
            'device_id', device_id
        )
    ) AS model_prediction,
    ai_query(
        endpoint => 'mistplay_fraud_demo_v6',
        request => named_struct(
            'account_id', account_id,
            'device_id', device_id
        )
    ).is_fraud_label AS predicted_fraud  -- Attempting to extract prediction
FROM ramin_serverless_aws_catalog.mistplay_fraud_demo.account_device
LIMIT 10